#Show base model

Load resnet model form pytorch model zoo

In [ ]:
from torchvision.models import resnet18
import os
import shutil

resnet_original = resnet18()
print(resnet_original)


# Helper method to run Tensorboard in Colab
def reinit_tensorboard(clear_log = True):
  # Directory for log files
  logs_base_dir = "runs"
  if clear_log:
    # Clear logs
    shutil.rmtree(logs_base_dir, ignore_errors = True)
    os.makedirs(logs_base_dir, exist_ok=True)
  # Colab magic
  %load_ext tensorboard
  %tensorboard --logdir {logs_base_dir}

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

# Show model structure
Do the same with your model when you create it.
Pay attention to spatial dimension of data in intermediate layers


In [ ]:
# Display model structure
# Do it for your model when 

import torch
from torch.utils.tensorboard import SummaryWriter

reinit_tensorboard()
writer = SummaryWriter(comment = "resnet")

dummy_input = torch.randn([1,3,224,224])
writer.add_graph(resnet_original, dummy_input)
writer.flush()
writer.close()


# Load the dataset

Load and preprocess the data. 
You can change size of datasets

In [ ]:
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader, random_split
import pickle

# https://github.com/facebookarchive/fb.resnet.torch/issues/180
cifar10_mean = (0.491, 0.482, 0.447)
cifar10_std = (0.247, 0.244, 0.262)


# Data preprocessing
transform=transforms.Compose([
                              transforms.ToTensor(), # PIL Image to Pytorch tensor
                              transforms.Normalize(cifar10_mean, cifar10_std) # https://pytorch.org/docs/stable/torchvision/transforms.html?highlight=transforms%20normalize#torchvision.transforms.Normalize
                              ])

dataset = datasets.CIFAR10("content", train=True, transform = transform ,  download=True)

# Split dataset into train and val
train_ds, val_ds, _= random_split(dataset, [10000, 1000 ,39000])

with open("content/cifar-10-batches-py/batches.meta",'rb') as infile:
  cifar_meta = pickle.load(infile)
labels = cifar_meta['label_names']


# Validation function. Don't change this code

import torch.optim as optim
def validate(model,testloader,device):
  correct = 0
  total = 0
  with torch.no_grad():
    for images, labels in testloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
  
  return correct / total  


# Code for training

Feel free to change hyperparameters.

In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

def train(model):
  writer = SummaryWriter(comment = "resnet")
  # Run model on cuda
  device = torch.device("cuda")
  
  model.train()
  model.to(device)

  train_loader = DataLoader(train_ds, batch_size = 128, shuffle = True)
  val_loader = DataLoader(val_ds, batch_size = 128, shuffle = False)

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4)
  best_accuracy = 0

  for epoch in range(30):
    for img_batch, labels_batch in train_loader:
      optimizer.zero_grad()
      output = model(img_batch.to(device))
      loss = criterion(output, labels_batch.to(device))
      loss.backward()
      optimizer.step()
      images = img_batch.cpu()
      label_nums = output.cpu()
    accuracy = validate(model,val_loader,device)
    if best_accuracy < accuracy:
      best_accuracy = accuracy
    writer.add_scalar('Accuracy',accuracy,epoch)
    writer.add_scalar('Loss/train',loss.cpu().item(),epoch)
    print("Epoch {} Loss {:.2f} Accuracy {:.2f}".format(epoch,loss.item(),accuracy))
    writer.flush()
  writer.close()


# Main task

Create your ouwn resnet - like model with depth of 15 -25 layers

Look at original paper before coding : https://arxiv.org/pdf/1512.03385.pdf
Don't directly use code from here: https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py

Train and test it on CIFAR10, compare results with original model

In [ ]:
import torch.nn as nn
import torch
import torch.nn.functional as F

'''
PDF файл из условия не открывается, нет доступа к сайту (видимо, сайт не работает)
'''

class Block(nn.Module):
    def __init__(self, in_channels, out_channels, downsample):
        super().__init__()
        self.shortcut = nn.Sequential()
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        if downsample:
            self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1)
            self.shortcut = nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=2), nn.BatchNorm2d(out_channels))

    def forward(self, input):
        shortcut = self.shortcut(input)
        input = nn.ReLU()(self.bn1(self.conv1(input)))
        input = nn.ReLU()(self.bn2(self.conv2(input)))
        input = input + shortcut
        return nn.ReLU()(input)

class CustomResnet(nn.Module):
    def __init__(self, input):
        super().__init__()
        self.layer0 = nn.Sequential(
            nn.Conv2d(input, 64, kernel_size=7, stride=2, padding=3),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.layer1 = nn.Sequential(Block(64, 64, downsample=False), Block(64, 64, downsample=False))
        self.layer2 = nn.Sequential(Block(64, 128, downsample=True), Block(128, 128, downsample=False))
        self.layer3 = nn.Sequential(Block(128, 256, downsample=True), Block(256, 256, downsample=False))
        self.layer4 = nn.Sequential(Block(256, 512, downsample=True), Block(512, 512, downsample=False))

    def forward(self, input):
        input = self.layer0(input)
        input = self.layer1(input)
        input = self.layer2(input)
        input = self.layer3(input)
        input = self.layer4(input)
        input = input.reshape(input.size(0), -1)
        input = self.fc(input)
        return input

#Train your model
 


In [ ]:
custom_resnet = CustomResnet(3)

In [ ]:
reinit_tensorboard()
writer = SummaryWriter(comment = "My custom resnet")
train_loader = DataLoader(train_ds, batch_size = 128, shuffle = True)
for img_batch, labels_batch in train_loader:
    writer.add_graph(custom_resnet, img_batch.to(torch.device("cpu")))
    break
writer.flush()
writer.close()

In [ ]:
train(custom_resnet)

Epoch 0 Loss 3.38 Accuracy 0.31
Epoch 1 Loss 1.52 Accuracy 0.39
Epoch 2 Loss 1.48 Accuracy 0.42
Epoch 3 Loss 2.19 Accuracy 0.43
Epoch 4 Loss 1.62 Accuracy 0.48
Epoch 5 Loss 1.78 Accuracy 0.50
Epoch 6 Loss 1.40 Accuracy 0.54
Epoch 7 Loss 1.31 Accuracy 0.55
Epoch 8 Loss 1.20 Accuracy 0.56
Epoch 9 Loss 1.31 Accuracy 0.58
Epoch 10 Loss 1.77 Accuracy 0.60
Epoch 11 Loss 1.34 Accuracy 0.60
Epoch 12 Loss 1.07 Accuracy 0.61
Epoch 13 Loss 1.00 Accuracy 0.62
Epoch 14 Loss 0.80 Accuracy 0.65
Epoch 15 Loss 1.52 Accuracy 0.62
Epoch 16 Loss 0.53 Accuracy 0.61
Epoch 17 Loss 0.89 Accuracy 0.62
Epoch 18 Loss 0.86 Accuracy 0.60
Epoch 19 Loss 0.55 Accuracy 0.62
Epoch 20 Loss 0.53 Accuracy 0.61
Epoch 21 Loss 1.24 Accuracy 0.62
Epoch 22 Loss 0.53 Accuracy 0.63
Epoch 23 Loss 0.41 Accuracy 0.59
Epoch 24 Loss 0.70 Accuracy 0.64
Epoch 25 Loss 0.11 Accuracy 0.63
Epoch 26 Loss 0.97 Accuracy 0.62
Epoch 27 Loss 0.44 Accuracy 0.62
Epoch 28 Loss 0.40 Accuracy 0.62
Epoch 29 Loss 0.56 Accuracy 0.63


#Train resnet18 from torchvision.models 

  Train from scratch Resnet18 on CIFAR-10
  Compare results with yours and with results from original paper:


 https://arxiv.org/pdf/1512.03385.pdf

In [ ]:
train(resnet_original)

Epoch 0 Loss 2.04 Accuracy 0.22
Epoch 1 Loss 1.74 Accuracy 0.35
Epoch 2 Loss 1.69 Accuracy 0.39
Epoch 3 Loss 1.48 Accuracy 0.44
Epoch 4 Loss 1.53 Accuracy 0.46
Epoch 5 Loss 1.93 Accuracy 0.52
Epoch 6 Loss 1.11 Accuracy 0.55
Epoch 7 Loss 1.21 Accuracy 0.54
Epoch 8 Loss 1.08 Accuracy 0.56
Epoch 9 Loss 1.43 Accuracy 0.58
Epoch 10 Loss 0.74 Accuracy 0.59
Epoch 11 Loss 1.07 Accuracy 0.62
Epoch 12 Loss 0.44 Accuracy 0.56
Epoch 13 Loss 0.58 Accuracy 0.62
Epoch 14 Loss 1.23 Accuracy 0.62
Epoch 15 Loss 1.47 Accuracy 0.61
Epoch 16 Loss 0.49 Accuracy 0.62
Epoch 17 Loss 0.58 Accuracy 0.64
Epoch 18 Loss 1.11 Accuracy 0.60
Epoch 19 Loss 0.33 Accuracy 0.61
Epoch 20 Loss 0.86 Accuracy 0.61
Epoch 21 Loss 0.66 Accuracy 0.58
Epoch 22 Loss 0.26 Accuracy 0.62
Epoch 23 Loss 0.11 Accuracy 0.62
Epoch 24 Loss 2.56 Accuracy 0.62
Epoch 25 Loss 0.96 Accuracy 0.61
Epoch 26 Loss 0.72 Accuracy 0.61
Epoch 27 Loss 0.46 Accuracy 0.62
Epoch 28 Loss 0.71 Accuracy 0.61
Epoch 29 Loss 0.69 Accuracy 0.62


#Place for conclusion:

Для resnet original наилучшее accuracy оказалось 0.64 (epoch = 17), при этом accuracy после 30 epoch стал равен 0.62

Для моей resnet наилучшее accuracy оказалось 0.65 (epoch = 14), при этом accuracy после 30 epoch стало 0.63

В целом, результаты очень близки, но у моей модели чуть-чуть лучше